Recommender system is an ML algorithm that combine several computational techniques to select personalized items based in user interesti.

The most common recommender system are:
* Content-based filtering System: produces recommendations that focus on item attributes and give recommendations based on similarities.
* Collaborative filtering System:  produces recommendations based on knowledge of attitudes toward items. This algorithm uses the "mass knowledge" to recommend items. Furthermore, collaborative filtering can be broken down into:
    * Collaborative memory filtering
    * Collaborative filtering by model

If you need more information about this technique, please check this [link](https://towardsdatascience.com/introduction-to-recommender-systems-1-971bd274f421)